## Line Parameters Estimation

### Load Package

In [1]:
using CSV, DataFrames
using Combinatorics

### Load Data

In [2]:
# network data
nodes = CSV.read(joinpath("..","data","NordicBaltic","nodes.csv"),DataFrame)
generators = CSV.read(joinpath("..","data","NordicBaltic","generators.csv"),DataFrame)
ehvacs = CSV.read(joinpath("..","data","NordicBaltic","existing_hvacs.csv"),DataFrame)
ehvdcs = CSV.read(joinpath("..","data","NordicBaltic","existing_hvdcs.csv"),DataFrame);

In [3]:
ehvacs

Row,id,name,f_node,f_node_name,t_node,t_node_name,f_area,f_area_name,t_area,t_area_name,existing_capacity,efficiency
,Int64,String15,Int64,String15,Int64,String7,Int64,String7,Int64,String7,Int64,Int64
1,1,link_NO_SE,7,Norway,6,Sweden,1,Nordic,1,Nordic,3787,1
2,2,line_SE_DK,6,Sweden,1,Denmark,1,Nordic,1,Nordic,507,1
3,3,line_LV_EE,4,Latvia,2,Estonia,2,Baltic,2,Baltic,1159,1
4,4,line_LT_LV,5,Lithuania,4,Latvia,2,Baltic,2,Baltic,910,1
5,5,line_FI_NO,3,Finland,7,Norway,1,Nordic,1,Nordic,120,1
6,6,line_FI_SE,3,Finland,6,Sweden,1,Nordic,1,Nordic,2607,1


### Helper function: getHaversineDistance()

In [4]:
function getHaversineDistance(latitude_1, longitude_1, latitude_2, longitude_2)
    R = 6371e3
    phi1 = latitude_1 * pi/180
    phi2 = latitude_2 * pi/180
    deltaphi = (latitude_2 - latitude_1) * pi/180
    deltalambda = (longitude_2 - longitude_1) * pi/180
    a = sin(deltaphi/2)^2 + cos(phi1) * cos(phi2) * sin(deltalambda/2)^2
    c = 2 * atan(sqrt(a), sqrt(1-a))
    d = R * c
    return d * 1e-3
end

getHaversineDistance (generic function with 1 method)

### Helper function: getFullCandidateLines()

In [5]:
function getFullCandidateLines(nodes,elines;ignore_area=true,hvdc=false,kv_nom=400)
    clines = Vector()
    if ignore_area
        clines = collect(Iterators.flatten(combinations(nodes.id,2)))
    else
        for a in unique(nodes.area)
            append!(clines, collect(Iterators.flatten(combinations(nodes[nodes.area.==a,:id],2))))
        end
    end
    clines = Array(reshape(clines,(2,:))')
    clines = DataFrame(clines,["f_node","t_node"])
    clines[!,:length] =  map(getHaversineDistance, 
        nodes[clines.f_node,:latitude], nodes[clines.f_node,:longitude], 
        nodes[clines.t_node,:latitude], nodes[clines.t_node,:longitude])
    if hvdc
        clines[!,:r] = clines.length .* 0.0378 ./ ((kv_nom*1e3)^2/100e6)
    else
        clines[!,:x] = clines.length .* 0.327 ./ ((kv_nom*1e3)^2/100e6)
    end
    clines[!,:existing_capacity] .= 0
    for (f,t) in zip(elines.f_node,elines.t_node)
        if isempty(clines[(clines.f_node.==f).&(clines.t_node.==t),:])
            if isempty(clines[(clines.f_node.==t).&(clines.t_node.==f),:])
            else
                clines[(clines.f_node.==t).&(clines.t_node.==f),:existing_capacity] .= 
                    elines[(elines.f_node.==f).&(elines.t_node.==t),:existing_capacity][1]
            end
        else
            clines[(clines.f_node.==f).&(clines.t_node.==t),:existing_capacity] .=
                elines[(elines.f_node.==f).&(elines.t_node.==t),:existing_capacity][1]
        end
    end
    clines.f_area = nodes[clines.f_node,:area]
    clines.t_area = nodes[clines.t_node,:area]
    clines
end

getFullCandidateLines (generic function with 1 method)

### Get full set of hvac candidate lines

In [7]:
chvacs = getFullCandidateLines(nodes,ehvacs;ignore_area=false,hvdc=false,kv_nom=380)
chvacs.f_area  = nodes[chvacs.f_node,:area]
chvacs

Row,f_node,t_node,length,x,existing_capacity,f_area,t_area
,Any,Any,Float64,Float64,Int64,Int64,Int64
1,1,3,1167.18,0.264314,0,1,1
2,1,6,296.174,0.0670699,507,1,1
3,1,7,544.403,0.123282,0,1,1
4,3,6,925.963,0.209688,2607,1,1
5,3,7,971.048,0.219898,120,1,1
6,6,7,605.283,0.137069,3787,1,1
7,2,4,184.043,0.0416772,1159,2,2
8,2,5,495.579,0.112226,0,2,2
9,4,5,322.054,0.0729304,910,2,2


### Get full set of hvdc candidate lines

In [8]:
chvdcs = getFullCandidateLines(nodes,ehvdcs;ignore_area=true,hvdc=true,kv_nom=380)

Row,f_node,t_node,length,r,existing_capacity,f_area,t_area
,Int64,Int64,Float64,Float64,Int64,Int64,Int64
1,1,2,1006.15,0.0263382,0,1,2
2,1,3,1167.18,0.0305537,0,1,1
3,1,4,982.022,0.0257067,0,1,2
4,1,5,885.507,0.0231802,0,1,2
5,1,6,296.174,0.00775304,0,1,1
6,1,7,544.403,0.014251,0,1,1
7,2,3,368.546,0.00964752,1017,2,1
8,2,4,184.043,0.00481773,0,2,2
9,2,5,495.579,0.0129729,0,2,2


### Generators data

In [9]:
first(generators,5)

Row,id,name_id,name,node,node_name,carrier,type,pmax,encost,rucost,rdcost,efficiency,pu_max,pu_min,area
,Int64,String31,String,Int64,String15,String15,String15,Float64,Float64,Float64,Float64,Float64,Int64,Float64,Int64
1,1,Esbjerg1_GAS,Esbjerg Reserve power station,1,Denmark,Gas,Gas,25.0,118.93,29.7325,29.7325,0.35,1,0.0,1
2,2,Grenaa1_GAS,Grenaa Reserve power station,1,Denmark,Gas,Gas,25.0,118.93,29.7325,29.7325,0.35,1,0.0,1
3,3,IRU_GAS,IRU Elektrijaam power station,2,Estonia,Gas,Gas,173.0,92.5,23.125,23.125,0.45,1,0.0,2
4,4,Lietuvos1_GAS,Lietuvos power station,5,Lithuania,Gas,Gas,300.0,118.93,29.7325,29.7325,0.35,1,0.0,2
5,5,Lietuvos2_GAS,Lietuvos power station,5,Lithuania,Gas,Gas,300.0,118.93,29.7325,29.7325,0.35,1,0.0,2


In [10]:
last(generators,5)

Row,id,name_id,name,node,node_name,carrier,type,pmax,encost,rucost,rdcost,efficiency,pu_max,pu_min,area
,Int64,String31,String,Int64,String15,String15,String15,Float64,Float64,Float64,Float64,Float64,Int64,Float64,Int64
1,75,Estonia_Other_RES,Estonia_Other_RES,2,Estonia,Other RES,Other RES,83.0,0.02,0.005,0.005,0.71,1,0.0,2
2,76,Finland_Other_RES,Finland_Other_RES,3,Finland,Other RES,Other RES,197.0,0.02,0.005,0.005,0.71,1,0.0,1
3,77,Latvia_Other_RES,Latvia_Other_RES,4,Latvia,Other RES,Other RES,36.0,0.02,0.005,0.005,0.71,1,0.0,2
4,78,Sweden_Other_RES,Sweden_Other_RES,6,Sweden,Other RES,Other RES,622.5,0.02,0.005,0.005,0.71,1,0.0,1
5,79,Norway_Other_RES,Norway_Other_RES,7,Norway,Other RES,Other RES,43.0,0.02,0.005,0.005,0.71,1,0.0,1
